# WIP # Chapter 2:

**Comprehensive Evaluation Strategies**

In [1]:
import os
import difflib
import Levenshtein
from nltk import word_tokenize
from nltk.translate import meteor
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from ranx import Qrels, Run, evaluate
from rouge import Rouge
from sklearn.metrics.pairwise import cosine_similarity
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import pandas as pd
import json
import pathlib
from typing import List, Dict, Any
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import nest_asyncio
nest_asyncio.apply()
import asyncio
from dotenv import load_dotenv
import numpy as np

load_dotenv()


True

## Building and improving an evaluation dataset

### Collecting data for evaluation
Get from data from the docs website [FAQs](https://docs.wandb.ai/guides/technical-faq) to test the system.

In [ ]:
# from datetime import datetime

# # TODO: Remove this once we more to the final project
# eval_artifact = wandb.Artifact(
#     name="eval_dataset",
#     type="dataset",
#     description="Evaluation dataset for RAG",
#     metadata={
#         "total_samples": 20,
#         "date_collected": datetime.now().strftime("%Y-%m-%d"),
#         "chapter": "Chapter 1",
#     },
# )
# eval_artifact.add_file("../data/eval/eval_dataset.jsonl")
# run.log_artifact(eval_artifact)

In [2]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 2",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: parambharat (rag-course). Use `wandb login --relogin` to force relogin


In [3]:
eval_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/eval_dataset:latest", type="dataset"
)
eval_dir = eval_artifact.download("../data/eval")
eval_dataset = pd.read_json(
    f"{eval_dir}/eval_dataset.jsonl", lines=True, orient="records"
)
eval_samples = eval_dataset.to_dict(orient="records")
eval_dataset

2024/07/03 11:15:34 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/rag-course/dev/okxu0mvh/artifact/938190750/wandb_manifest.json?Expires=1719989134&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=lhO4D3O9Lvd%2BytjLLF8RwTgRaf5%2BlhFgxiNilPlSNGxaSN9Cpqa0p8kcmRLcqG9tZJh3mYqty%2BGo7IRG8qJzxNdCyjLUBf9OFbEOh9DrXK2i7fZYTJ5THqBbHyDyrIFb9dyeY5r3qVWJF6JQJXzucW%2BhnIKu7OcrLu36fJSoArD%2FpwHZEdpJI4rwz5eX4Z0NP6n0PNyEHXR6lZ3b3eh8cQMKubQaZgp%2BuaGXXMfTXXzL6EAuhtquapwBjeGB6xryHh6vPnNCawW0htoF7svt2lXcIEr32mVrbh6l4IKFosFeRZeirNyuw6Hl8HGBxdVvvSnfFcDaDaYGuN6iZCEdwQ%3D%3D


,question,answer,source
0,What is the difference between `.log()` and `....,The summary is the value that shows in the tab...,guides/technical-faq/general.md
1,How do I switch between accounts on the same m...,If you have two W&B accounts working from the ...,guides/technical-faq/general.md
2,How is W&B different from TensorBoard?,"We love the TensorBoard folks, and we have a T...",guides/technical-faq/general.md
3,What is the difference between team and organi...,A team is a collaborative workspace for a grou...,guides/technical-faq/admin.md
4,What is the difference between team and entity...,A team is a collaborative workspace for a grou...,guides/technical-faq/admin.md
5,"Can I just log metrics, no code or dataset exa...","**Dataset Examples**\n\nBy default, we don't l...",guides/technical-faq/metrics-and-performance.md
6,How can I log a metric that doesn't change ove...,Using `wandb.log({'final_accuracy': 0.9}` will...,guides/technical-faq/metrics-and-performance.md
7,How many runs to create per project?,We recommend you have roughly 10k runs per pro...,guides/technical-faq/metrics-and-performance
8,Can I run wandb offline?,If you're training on an offline machine and w...,guides/technical-faq/setup.md
9,How do I deal with network issues?,If you're seeing SSL or network errors:`wandb:...,guides/technical-faq/troubleshooting.md


### Evaluating the Retriever

This is a search problem, it's easiest to start with tradiaional Information retrieval metrics.


ref: https://weaviate.io/blog/retrieval-evaluation-metrics

**TODO** Add weave model and evals in this section

In [4]:
# Reload the data from Chapter 1
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

2024/07/03 11:15:40 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/rag-course/dev/0z2t11h3/artifact/936065852/wandb_manifest.json?Expires=1719989140&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=JTlCAuLgo9flPZRV6xn0UMyUODR%2FFQ0HCVccTX%2F57%2Flp%2FgJd837C30XMJw45HXMHr2dUi5Q2kZg%2F6blnoZ2Rd3RoLADc8G%2BH8oA8bilHkIIPh1m0EanO6%2B%2FdB4Ywe4K0t7wg06c6FYa8VjkFOXtzlNu%2BWWBPndsuQ8XPClXITzjbKUJpWPzTxP6ItWvkVVCmbVB2ZZdki2bNAkiExE1%2BT%2FGIc9nmviBMB%2B1eBeCFnQ4diLTK42BmTbnsu7dcUOar7b%2B5sNNeUA3TTWVhStCdAisntdspZDan7pNIXi8p6esmvV4wL%2FibhmgSikgKpGJp61lPFEGLNUQNtr0ZEjnHtA%3D%3D


[{'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log results qui

In [5]:
# Reuse the Retriever class from Chapter 1
class Retriever:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.index = None
        self.data = None

    def index_data(self, data):
        self.data = data
        docs = [doc["cleaned_content"] for doc in data]
        self.index = self.vectorizer.fit_transform(docs)

    #@weave.op
    def search(self, query, k=5):
        query_vec = self.vectorizer.transform([query])
        cosine_distances = cdist(
            query_vec.todense(), self.index.todense(), metric="cosine"
        )[0]
        top_k_indices = cosine_distances.argsort()[:k]
        output = []
        for idx in top_k_indices:
            output.append(
                {
                    "source": self.data[idx]["metadata"]["source"],
                    "text": self.data[idx]["cleaned_content"],
                    "score": 1 - cosine_distances[idx],
                }
            )
        return output

In [6]:
retriever = Retriever()
retriever.index_data(chunked_data)

In [7]:
def calculate_hit_rate(retrieved_docs: List[str], actual_doc: str) -> float:
    """
    Calculate the hit rate for a single query.

    :param retrieved_docs: List of retrieved documents
    :param actual_doc: The single actual relevant document
    :return: Hit rate (1 if the relevant document is retrieved, 0 otherwise)
    """
    return 1 if actual_doc in retrieved_docs else 0


In [8]:
hit_rates = []
for sample in eval_samples:
    query = sample["question"]
    expected_source = sample["source"]
    search_results = [doc['source'] for doc in retriever.search(query, k=5)]
    hit_rate = calculate_hit_rate(search_results, expected_source)
    hit_rates.append({"query": query, "hit_rate": hit_rate})

hit_rate_df = pd.DataFrame(hit_rates)
display(hit_rate_df)

,query,hit_rate
0,What is the difference between `.log()` and `....,1
1,How do I switch between accounts on the same m...,1
2,How is W&B different from TensorBoard?,1
3,What is the difference between team and organi...,1
4,What is the difference between team and entity...,1
5,"Can I just log metrics, no code or dataset exa...",1
6,How can I log a metric that doesn't change ove...,1
7,How many runs to create per project?,0
8,Can I run wandb offline?,0
9,How do I deal with network issues?,1


In [9]:
# we need a single number to rate the retrieval system
# the mean hit rate is a good metric to evaluate the retrieval system as a whole

print(f"Mean Hit Rate: {hit_rate_df['hit_rate'].mean():.4f}")
print(f"Std-dev Hit Rate: {hit_rate_df['hit_rate'].std():.4f}")

Mean Hit Rate: 0.7000
Std-dev Hit Rate: 0.4702


In [10]:
# MRR (Mean Reciprocal Rank) is a metric that measures the quality of the retrieval system by evaluating the proportion of queries for which the most relevant document is retrieved.
# Let's calculate the MRR score for our retrieval system

def calculate_mrr(retrieved_docs: List[str], actual_doc: str) -> float:
    mrr_score = 0
    for rank, result in enumerate(retrieved_docs, 1):
        if result == actual_doc:
            mrr_score = 1 / rank
            break
    return mrr_score

In [11]:
mrr_scores = []
for sample in eval_samples:
    query = sample["question"]
    expected_source = sample["source"]
    search_results = [doc['source'] for doc in retriever.search(query, k=5)]
    mrr_score = calculate_mrr(search_results, expected_source)
    mrr_scores.append({"query": query, "mrr_score": mrr_score})

mrr_scores_df = pd.DataFrame(mrr_scores)
display(mrr_scores_df)

,query,mrr_score
0,What is the difference between `.log()` and `....,0.500000
1,How do I switch between accounts on the same m...,1.000000
2,How is W&B different from TensorBoard?,0.333333
3,What is the difference between team and organi...,1.000000
4,What is the difference between team and entity...,1.000000
5,"Can I just log metrics, no code or dataset exa...",1.000000
6,How can I log a metric that doesn't change ove...,0.500000
7,How many runs to create per project?,0.000000
8,Can I run wandb offline?,0.000000
9,How do I deal with network issues?,1.000000


In [12]:
# we need a single number to rate the retrieval system
# the mean mrr score is a good metric to evaluate the retrieval system
print(f"Mean MRR Score: {mrr_scores_df['mrr_score'].mean():.4f}")

# Looking at the mean might not give us the complete picture. We can also look at the distribution of the MRR scores
print("MRR Score Statistics:")
display(pd.DataFrame(mrr_scores_df["mrr_score"].describe()).T)

Mean MRR Score: 0.5083
MRR Score Statistics:


,count,mean,std,min,25%,50%,75%,max
mrr_score,20.0,0.508333,0.417017,0.0,0.0,0.5,1.0,1.0


#### Evaluating retrieval on other metrics

In [13]:
# Similarly, we can also evaluate the retrieval system on other metrics
# Writing these might be tedious, but we can use the `ranx` library to evaluate the retrieval system
# Metrics Include
# NDCG (Normalized Discounted Cumulative Gain)
# MAP (Mean Average Precision)
# Hit Rate
# Precision
# Recall
# F1 Score


RETRIEVAL_METRICS = ["ndcg@5", "map@5", "mrr", "hit_rate", "precision", "recall", "f1"]


def evaluate_retriever(retrieved_docs: List[Dict[str, Any]], actual_doc: str) -> Dict[str, Any]:
    qrels = Qrels({"query": {actual_doc: 1}})
    run = Run({"query": {doc["source"]: doc["score"] for doc in retrieved_docs}})
    return evaluate(qrels, run, metrics=RETRIEVAL_METRICS)


In [14]:
retrieval_scores = []
for sample in tqdm(eval_samples):
    query = sample["question"]
    expected_source = sample["source"]
    search_results = retriever.search(query, k=5)
    eval_scores = evaluate_retriever(search_results, expected_source)
    retrieval_scores.append({"query": query, **eval_scores})

retrieval_scores_df = pd.DataFrame(retrieval_scores)
display(retrieval_scores_df)

100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


,query,ndcg@5,map@5,mrr,hit_rate,precision,recall,f1
0,What is the difference between `.log()` and `....,0.63093,0.500000,0.500000,1.0,0.250000,1.0,0.400000
1,How do I switch between accounts on the same m...,1.00000,1.000000,1.000000,1.0,0.200000,1.0,0.333333
2,How is W&B different from TensorBoard?,0.50000,0.333333,0.333333,1.0,0.333333,1.0,0.500000
3,What is the difference between team and organi...,1.00000,1.000000,1.000000,1.0,0.250000,1.0,0.400000
4,What is the difference between team and entity...,1.00000,1.000000,1.000000,1.0,0.200000,1.0,0.333333
5,"Can I just log metrics, no code or dataset exa...",1.00000,1.000000,1.000000,1.0,0.250000,1.0,0.400000
6,How can I log a metric that doesn't change ove...,0.63093,0.500000,0.500000,1.0,0.200000,1.0,0.333333
7,How many runs to create per project?,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
8,Can I run wandb offline?,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
9,How do I deal with network issues?,1.00000,1.000000,1.000000,1.0,0.200000,1.0,0.333333


In [15]:
print("\nMean Overall Retrieval Scores:")
display(pd.DataFrame(retrieval_scores_df[RETRIEVAL_METRICS].mean()).T)

print("\nOverall Retrieval Score Statistics:")
display(pd.DataFrame(retrieval_scores_df[RETRIEVAL_METRICS].describe()).T)


Mean Overall Retrieval Scores:


,ndcg@5,map@5,mrr,hit_rate,precision,recall,f1
0,0.557732,0.508333,0.508333,0.7,0.161667,0.7,0.261667



Overall Retrieval Score Statistics:


,count,mean,std,min,25%,50%,75%,max
ndcg@5,20.0,0.557732,0.414796,0.0,0.0,0.630930,1.00,1.000000
map@5,20.0,0.508333,0.417017,0.0,0.0,0.500000,1.00,1.000000
mrr,20.0,0.508333,0.417017,0.0,0.0,0.500000,1.00,1.000000
hit_rate,20.0,0.700000,0.470162,0.0,0.0,1.000000,1.00,1.000000
precision,20.0,0.161667,0.113181,0.0,0.0,0.200000,0.25,0.333333
recall,20.0,0.700000,0.470162,0.0,0.0,1.000000,1.00,1.000000
f1,20.0,0.261667,0.180407,0.0,0.0,0.333333,0.40,0.500000


### Using an LLM as a Retrieval Judge

**ref: https://arxiv.org/pdf/2406.06519**

How do we evaluate if we don't have any ground truth? 

We can use a powerful LLM as a judge to evaluate the retriever. 


In [16]:

RETRIEVAL_EVAL_PROMPT ="""
Given a query and a document excerpt, you must provide a score on an integer scale of 0 to 3 with the following meanings:
    0 = represent that the excerpt has nothing to do with the query,
    1 = represents that the excerpt seems related to the query but does not help answer it,
    2 = represents that the excerpt has some answer for the query, but the answer may be a bit unclear, or hidden amongst extraneous information and
    3 = represents that the excerpt is dedicated to the query and contains the exact answer.

Important Instruction: Assign category 1 if the excerpt is somewhat related to the topic but not completely, category 2 if excerpt presents something very important related to the entire topic but also has some extra information and category 3 if the excerpt only and entirely refers to the topic. If none of the above satisfies give it category 0.

Query: {query}
Document: {document}

Split this problem into steps:
Consider the underlying intent of the query. Measure how well the content matches a likely intent of the query(M).
Measure how trustworthy the excerpt is (T).
Consider the aspects above and the relative importance of each, and decide on a final score (O). 
Final score must be an integer value only.
Do not provide any code in result. Provide each score in the following JSON format: 

```json
{{"final_score": <integer score without providing any reasoning.>}}
```
"""

In [17]:
client = AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"])

async def evaluate_retriever_using_llm_judge(query: str, passage: str) -> int:
    response = await client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": RETRIEVAL_EVAL_PROMPT.format(query=query, passage=passage)}],
        response_format={"type": "json_object"}
    )
    return response.choices[0].message.content


In [18]:
async def run_retriever_evaluation_using_llm(eval_samples):
    scores = []
    for sample in eval_samples:
        query = sample["question"]
        search_results = retriever.search(query, k=5)
        tasks = []
        for result in search_results:
            tasks.append(evaluate_retriever_using_llm_judge(query, result["text"]))
        sample_scores = await asyncio.gather(*tasks)
        sample_scores = map(json.loads, sample_scores)
        sample_scores = list(map(lambda x: x["final_score"], sample_scores))
        scores.append({"query": query, "scores": sample_scores})
    return scores
    

In [19]:
llm_judge_retrieval_results = asyncio.run(run_retriever_evaluation_using_llm(eval_samples))

In [34]:
# we have the scores for each document
llm_judge_retrieval_results_df = pd.DataFrame(llm_judge_retrieval_results)

# we can compute the reciprocal rank of the first document that is relevant to the query i.e. rated as 3 by our llm judge.
def compute_rank_score(scores: List[int]) -> float:
    rank_score = 0
    for rank, result in enumerate(scores, 1):
        if result == 3:
            rank_score = 1 / rank
            return rank_score
    return rank_score

llm_judge_retrieval_results_df["rank_score"] = llm_judge_retrieval_results_df["scores"].map(compute_rank_score)


display(llm_judge_retrieval_results_df)


print(f"Mean Rank Score: {llm_judge_retrieval_results_df['rank_score'].mean():.4f}")
print(f"Std-dev Rank Score: {llm_judge_retrieval_results_df['rank_score'].std():.4f}")
    

,query,scores,rank_score
0,What is the difference between `.log()` and `....,"[1, 3, 0, 2, 0]",0.500000
1,How do I switch between accounts on the same m...,"[3, 0, 0, 0, 0]",1.000000
2,How is W&B different from TensorBoard?,"[1, 1, 3, 3, 2]",0.333333
3,What is the difference between team and organi...,"[3, 1, 0, 0, 0]",1.000000
4,What is the difference between team and entity...,"[3, 0, 2, 0, 1]",1.000000
5,"Can I just log metrics, no code or dataset exa...","[3, 1, 2, 2, 1]",1.000000
6,How can I log a metric that doesn't change ove...,"[0, 3, 2, 3, 1]",0.500000
7,How many runs to create per project?,"[3, 0, 0, 0, 1]",1.000000
8,Can I run wandb offline?,"[3, 3, 3, 2, 2]",1.000000
9,How do I deal with network issues?,"[2, 0, 1, 0, 0]",0.000000


Mean Rank Score: 0.7250
Std-dev Rank Score: 0.3301


## Evaluating the Response

In [21]:
# Lets reload the Response Generator and the RAGPipeline from the previous chapter
class ResponseGenerator:
    def __init__(self, model: str, prompt: str):
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        self.model = model
        self.prompt = prompt

    def generate_context(self, context: List[Dict[str, any]]) -> str:
        return "\n".join(
            [f"Source: {item['source']}\nText: {item['text']}\n\n" for item in context]
        )

    # @weave.op()
    def generate_response(self, query: str, context: List[Dict[str, any]]) -> str:
        context_text = self.generate_context(context)
        system_message = {
            "role": "system",
            "content": self.prompt.format(context=context_text),
        }
        user_message = {"role": "user", "content": f"Question: {query}\n\nAnswer:"}
        response = self.client.chat.completions.create(
            model=self.model, messages=[system_message, user_message]
        )
        return response.choices[0].message.content
    

PROMPT = (
    "You are a helpful customer support assistant that can answer questions about W&B\n\n"
    "Your answers must be based only on the provided context.\n\n"
    "<context>\n{context}\n</context>"
)
response_generator = ResponseGenerator(model="gpt-3.5-turbo", prompt=PROMPT)

class RAGPipeline:
    def __init__(self, retriever: Retriever, response_generator: ResponseGenerator, top_k: int = 5):
        self.retriever = retriever
        self.response_generator = response_generator
        self.top_k = top_k

    
    # @weave.op
    def __call__(self, query: str) -> str:
        context = self.retriever.search(query, self.top_k)
        return self.response_generator.generate_response(query, context)

In [22]:
# We can measure the similarity of the response to the expected answer using difflib and Levenshtein distance
# These are simple metrics.

def calculate_diff_score(candidate, reference):
    return difflib.SequenceMatcher(None, candidate, reference).ratio()


def calculate_levenshtein_score(candidate, reference):
    return Levenshtein.ratio(candidate, reference)



# semantic answer similarity. (SAS) - https://arxiv.org/abs/2108.06130
# Originally, one should use a transformer based cross-encoder to measure and classify this. 
# For example, use something from https://sbert.net/docs/cross_encoder/usage/usage.html
# we can also calculate the cosine similarity between the candidate and the reference using our retriever's vectorizer
def calculate_similarity(candidate, reference):
    vectors = retriever.vectorizer.transform([candidate, reference])
    similarity = cosine_similarity(vectors)[0][1]
    return similarity




# or we can use traditional metrics used to measure generation systems.
# ref: https://blog.paperspace.com/automated-metrics-for-evaluating-generated-text/

def calculate_rouge(candidate, reference):
    rouge = Rouge(metrics=["rouge-l"], stats="f")
    scores = rouge.get_scores(candidate, reference)
    return scores[0]["rouge-l"]["f"]


def calculate_bleu(candidate, reference):
    chencherry = SmoothingFunction()
    smoothing_function = chencherry.method2

    reference = word_tokenize(reference)
    candidate = word_tokenize(candidate)
    score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)
    return score


def calculate_meteor(candidate, reference):
    reference = word_tokenize(reference)
    candidate = word_tokenize(candidate)
    meteor_score = meteor([candidate], reference)
    return meteor_score


In [23]:
rag_pipeline = RAGPipeline(retriever, response_generator)

response_scores = []
for sample in tqdm(eval_samples):
    query = sample['question']
    actual_answer = rag_pipeline(query)
    expected_answer = sample['answer']
    diff_score = calculate_diff_score(actual_answer, expected_answer)
    levenshtein_score = calculate_levenshtein_score(actual_answer, expected_answer)
    rouge_score = calculate_rouge(actual_answer, expected_answer)
    bleu_score = calculate_bleu(actual_answer, expected_answer)
    meteor_score = calculate_meteor(actual_answer, expected_answer)
    similarity_score = calculate_similarity(actual_answer, expected_answer)

    response_scores.append({
        "query": query,
        "expected_answer": expected_answer,
        "actual_answer": actual_answer,
        "diff_score": diff_score,
        "levenshtein_score": levenshtein_score,
        "rouge_score": rouge_score,
        "bleu_score": bleu_score,
        "meteor_score": meteor_score,
        "similarity_score": similarity_score
    })

100%|██████████| 20/20 [00:37<00:00,  1.88s/it]


In [24]:

response_scores_df = pd.DataFrame(response_scores)
display(response_scores_df)

GENERATION_METRICS = [col for col in response_scores_df.columns if "score" in col]


print("\nMean Overall Generation Scores:")
display(pd.DataFrame(response_scores_df[GENERATION_METRICS].mean()).T)

print("\nOverall Generation Score Statistics:")
display(pd.DataFrame(response_scores_df[GENERATION_METRICS].describe()).T)

,query,expected_answer,actual_answer,diff_score,levenshtein_score,rouge_score,bleu_score,meteor_score,similarity_score
0,What is the difference between `.log()` and `....,The summary is the value that shows in the tab...,The summary is the value that shows in the tab...,0.156752,0.613787,0.517241,0.249934,0.557228,0.723108
1,How do I switch between accounts on the same m...,If you have two W&B accounts working from the ...,To switch between accounts on the same machine...,0.417234,0.603175,0.722689,0.564265,0.806683,0.858553
2,How is W&B different from TensorBoard?,"We love the TensorBoard folks, and we have a T...",Weights & Biases (W&B) differs from TensorBoar...,0.251166,0.540210,0.384615,0.163461,0.592270,0.490417
3,What is the difference between team and organi...,A team is a collaborative workspace for a grou...,A team is a collaborative workspace for a grou...,1.000000,1.000000,1.000000,1.000000,0.999992,1.000000
4,What is the difference between team and entity...,A team is a collaborative workspace for a grou...,A team is a collaborative workspace for a grou...,0.624762,0.815238,0.729730,0.662632,0.871998,0.921223
5,"Can I just log metrics, no code or dataset exa...","**Dataset Examples**\n\nBy default, we don't l...","By default, W&B doesn't log any dataset exampl...",0.416481,0.665924,0.512397,0.228351,0.636014,0.775382
6,How can I log a metric that doesn't change ove...,Using `wandb.log({'final_accuracy': 0.9}` will...,Using `wandb.log({'final_accuracy': 0.9})` wil...,0.788382,0.804979,0.655172,0.605345,0.726232,0.896947
7,How many runs to create per project?,We recommend you have roughly 10k runs per pro...,We recommend creating roughly 10k runs per pro...,0.872727,0.872727,0.769231,0.527979,0.777745,0.860045
8,Can I run wandb offline?,If you're training on an offline machine and w...,"Yes, you can run `wandb` in offline mode. To r...",0.070252,0.450055,0.220339,0.046529,0.391516,0.507570
9,How do I deal with network issues?,If you're seeing SSL or network errors:`wandb:...,If you are experiencing SSL or network errors ...,0.286316,0.676491,0.581818,0.389579,0.676808,0.793544



Mean Overall Generation Scores:


,diff_score,levenshtein_score,rouge_score,bleu_score,meteor_score,similarity_score
0,0.4883,0.66825,0.594625,0.437911,0.666054,0.747174



Overall Generation Score Statistics:


,count,mean,std,min,25%,50%,75%,max
diff_score,20.0,0.488300,0.315345,0.061971,0.247160,0.391636,0.804537,1.000000
levenshtein_score,20.0,0.668250,0.212530,0.162728,0.537269,0.671030,0.829610,1.000000
rouge_score,20.0,0.594625,0.225095,0.220339,0.445287,0.617832,0.739605,1.000000
bleu_score,20.0,0.437911,0.282360,0.000107,0.212128,0.441510,0.619667,1.000000
meteor_score,20.0,0.666054,0.201280,0.332695,0.536610,0.656411,0.816121,0.999992
similarity_score,20.0,0.747174,0.182418,0.469942,0.583945,0.784463,0.903016,1.000000


### Using an LLM as a Response Judge

Some metrics cannot be defined objectively and are particularly useful for more subjective or complex criteria.
We care about correctness, faithfulness, and relevance.

- **Answer Correctness** - Is the generated answer correct compared to the reference and thoroughly answers the user's query?
- **Answer Relevancy** - Is the generated answer relevant and comprehensive?
- **Answer Factfulness** - Is the generated answer factually consistent with the context document?


In [25]:

CORRECTNESS_EVAL_PROMPT ="""
You are a Weight & Biases support expert tasked with evaluating the correctness of answers to questions asked by users to a technical support chatbot. 
You are tasked with judging the correctness of a generated answer based on the user's query, and a reference answer.

You will be given the following information:

<query>
{query}
</query>

<reference_answer>
{reference_answer}
</reference_answer>

<generated_answer>
{generated_answer}
</generated_answer>

Important Instruction: To evaluate the generated answer, follow these steps:

1. Intent Analysis: Consider the underlying intent of the query.
2. Relevance: Check if the generated answer addresses all aspects of the question.
3. Accuracy: Compare the generated answer to the reference answer for completeness and correctness.
4. Trustworthiness: Measure how trustworthy the generated answer is when compared to the reference.

Assign a score on an integer scale of 0 to 3 with the following meanings:
- 0 = The generated answer is incorrect and does not satisfy any of the criteria.
- 1 = The generated answer is partially correct, contains mistakes or is not factually correct.
- 2 = The generated answer is correct but includes some extra information, is incomplete or misses some evaluation criteria.
- 3 = The generated answer is correct, completely answers the query, does not contain any mistakes, and is factually consistent with the reference answer.

After your analysis, provide your verdict in the following JSON format:

```json
{{
    "reason": "<<Provide a brief explanation for your decision here>>",
    "final_score": <<Provide a score as per the above guidelines>>,
    "decision": "<<Provide your final decision here, either 'correct' or 'incorrect'>>"
}}
```

Here are some examples of correct output:

Example 1:
```json
{{
    "reason": "The generated answer has the exact details as the reference answer and completely answers the user's query.",
    "final_score": 3,
    "decision": "correct"
}}
```

Example 2:
```json
{{
    "reason": "The generated answer doesn't match the reference answer and deviates from the user's query.",
    "final_score": 0,
    "decision": "incorrect"
}}
```

Example 3:
```json
{{
    "reason": "The generated answer follows the same steps as the reference answer. However, it includes assumptions about functions that are not requested in the user's query",
    "final_score": 2,
    "decision": "correct"
}}
```

Example 4:
```json
{{
    "reason": "The generated answer is incorrect, irrelevant, and not factually correct and completely misses the user's intent.",
    "final_score": 0,
    "decision": "incorrect"
}}
```

Please provide your evaluation based on the given information and format your response according to the specified JSON structure.
"""

In [26]:
client = AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"])

async def evaluate_correctness_using_llm_judge(query: str, reference_answer: str, generated_answer: str) -> int:
    response = await client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": CORRECTNESS_EVAL_PROMPT.format(query=query, reference_answer=reference_answer, generated_answer=generated_answer)}],
        response_format={"type": "json_object"}
    )
    return response.choices[0].message.content


In [27]:
async def run_correctness_evaluation_using_llm(response_scores):
    tasks = []
    for row in response_scores:
        query = row["query"]
        expected_answer = row["expected_answer"]
        generated_answer = row["actual_answer"]
        tasks.append(evaluate_correctness_using_llm_judge(query, expected_answer, generated_answer))
    scores = await asyncio.gather(*tasks)
    scores = list(map(json.loads, scores))
    return scores
    

In [28]:
llm_judge_correctness_results = asyncio.run(run_correctness_evaluation_using_llm(response_scores))

In [30]:
correctness_eval_df = pd.DataFrame(llm_judge_correctness_results)
response_evals_df = pd.concat([response_scores_df, correctness_eval_df], axis=1)
response_evals_df

,query,expected_answer,actual_answer,diff_score,levenshtein_score,rouge_score,bleu_score,meteor_score,similarity_score,reason,final_score,decision
0,What is the difference between `.log()` and `....,The summary is the value that shows in the tab...,The summary is the value that shows in the tab...,0.156752,0.613787,0.517241,0.249934,0.557228,0.723108,The generated answer correctly addresses the m...,2,correct
1,How do I switch between accounts on the same m...,If you have two W&B accounts working from the ...,To switch between accounts on the same machine...,0.417234,0.603175,0.722689,0.564265,0.806683,0.858553,The generated answer closely follows the refer...,3,correct
2,How is W&B different from TensorBoard?,"We love the TensorBoard folks, and we have a T...",Weights & Biases (W&B) differs from TensorBoar...,0.251166,0.540210,0.384615,0.163461,0.592270,0.490417,The generated answer closely follows the refer...,3,correct
3,What is the difference between team and organi...,A team is a collaborative workspace for a grou...,A team is a collaborative workspace for a grou...,1.000000,1.000000,1.000000,1.000000,0.999992,1.000000,The generated answer is identical to the refer...,3,correct
4,What is the difference between team and entity...,A team is a collaborative workspace for a grou...,A team is a collaborative workspace for a grou...,0.624762,0.815238,0.729730,0.662632,0.871998,0.921223,The generated answer accurately mirrors the re...,3,correct
5,"Can I just log metrics, no code or dataset exa...","**Dataset Examples**\n\nBy default, we don't l...","By default, W&B doesn't log any dataset exampl...",0.416481,0.665924,0.512397,0.228351,0.636014,0.775382,The generated answer closely matches the refer...,3,correct
6,How can I log a metric that doesn't change ove...,Using `wandb.log({'final_accuracy': 0.9}` will...,Using `wandb.log({'final_accuracy': 0.9})` wil...,0.788382,0.804979,0.655172,0.605345,0.726232,0.896947,The generated answer is mostly correct but con...,1,incorrect
7,How many runs to create per project?,We recommend you have roughly 10k runs per pro...,We recommend creating roughly 10k runs per pro...,0.872727,0.872727,0.769231,0.527979,0.777745,0.860045,The generated answer has the exact details as ...,3,correct
8,Can I run wandb offline?,If you're training on an offline machine and w...,"Yes, you can run `wandb` in offline mode. To r...",0.070252,0.450055,0.220339,0.046529,0.391516,0.507570,"The generated answer is generally correct, men...",2,correct
9,How do I deal with network issues?,If you're seeing SSL or network errors:`wandb:...,If you are experiencing SSL or network errors ...,0.286316,0.676491,0.581818,0.389579,0.676808,0.793544,The generated answer follows the intent of the...,2,correct


In [ ]:
response_scores_df["correctness_score"] = llm_judge_correctness_results
display(response_scores_df)

## Exercise

1. Implement the `Relevance` and `Faithfulness` evaluators and evaluate the pipeline on all the dimensions.
2. Generate and share a W&B report with the following sections in the form of tables and charts:
    
    - Summary of the evaluation
    - Retreival Evaluations
        - IR Metrics
        - LLM As a Retrieval Judge Metric
    - Response Evalations
        - Traditional NLP Metrics
        - LLM Judgement Metrics
    - Overall Evalations
    - Conclusion
